In [62]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import glob
import sys
import os
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../src')
from clf_parser import CLFParse

In [64]:
# Modify path here for your particular system
data_dir = Path('/Users/kaipak/datasets/pds/pds_logs/report_service/logs/final/rings/rings-apache-metrics')
tools2_logs = sorted(glob.glob(str(data_dir) + "/tools2*"))
server1_logs = sorted(glob.glob(str(data_dir) + "/server1*"))

In [65]:
parser_tools2 = CLFParse(tools2_logs)
parser_server1 = CLFParse(server1_logs)

In [66]:
%%time
# This might take a while. 
parser_tools2.parse_files()

Logfiles list contains 198 files.
10 files have been processed. 0.03 minutes elapsed
20 files have been processed. 0.12 minutes elapsed
30 files have been processed. 0.16 minutes elapsed
40 files have been processed. 0.22 minutes elapsed
50 files have been processed. 0.3 minutes elapsed
60 files have been processed. 0.38 minutes elapsed
70 files have been processed. 0.48 minutes elapsed
80 files have been processed. 0.59 minutes elapsed
90 files have been processed. 0.7 minutes elapsed
100 files have been processed. 0.8 minutes elapsed
110 files have been processed. 0.89 minutes elapsed
120 files have been processed. 1.01 minutes elapsed
130 files have been processed. 1.11 minutes elapsed
140 files have been processed. 1.2 minutes elapsed
150 files have been processed. 1.3 minutes elapsed
160 files have been processed. 1.4 minutes elapsed
170 files have been processed. 1.51 minutes elapsed
180 files have been processed. 1.61 minutes elapsed
190 files have been processed. 1.69 minutes e

In [67]:
parser_tools2.to_file(type="feather", filepath="../datasets/rings_tools2.feather")

Completed creating feather file at ../datasets/rings_tools2.feather.


In [68]:
%%time
# This might take a while. 
parser_server1.parse_files()

Logfiles list contains 197 files.
10 files have been processed. 0.14 minutes elapsed
20 files have been processed. 0.29 minutes elapsed
30 files have been processed. 0.54 minutes elapsed
40 files have been processed. 0.67 minutes elapsed
50 files have been processed. 0.86 minutes elapsed
60 files have been processed. 1.08 minutes elapsed
70 files have been processed. 1.27 minutes elapsed
80 files have been processed. 1.49 minutes elapsed
90 files have been processed. 1.76 minutes elapsed
100 files have been processed. 1.94 minutes elapsed
110 files have been processed. 2.12 minutes elapsed
120 files have been processed. 2.46 minutes elapsed
130 files have been processed. 2.65 minutes elapsed
140 files have been processed. 2.88 minutes elapsed
150 files have been processed. 3.2 minutes elapsed
160 files have been processed. 3.42 minutes elapsed
170 files have been processed. 3.61 minutes elapsed
180 files have been processed. 3.91 minutes elapsed
190 files have been processed. 4.15 minu

In [69]:
parser_server1.to_file(type="feather", filepath="../datasets/rings_server1.feather")

Completed creating feather file at ../datasets/rings_server1.feather.


## Example Read in File and Create Data Viz


In [ ]:
%%time
df_server1 = pd.read_parquet('../datasets/rings_server1.pq')

In [ ]:
df_server1.shape

In [ ]:
df_server1.head()

In [ ]:
df_summary = (
    df_server1.
    groupby(by='status').
    size().to_frame('requests').reset_index().
    sort_values(by='requests', ascending=False)
)
fig, ax = plt.subplots(figsize=(15, 12))
bar_p = sns.barplot(data=df_summary, x='status', y='requests', 
                    order=df_summary.sort_values('requests', ascending=False).status)

In [ ]:
df_summary

In [ ]:
df_server1.shape